## Establish  data base connection and import required libraries

In [1]:
#!/usr/bin/python

import   pandas          as     pd
import   numpy           as     np

import   MySQLdb
import   mysql.connector
import   re

from     sqlalchemy      import create_engine
from     bs4             import BeautifulSoup
import   webbrowser
from     prettytable     import PrettyTable

In [2]:
def get_matched_profiles(user_age, pref_age_l, pref_age_u):
    pref_male_df  = male_df.loc[(male_df['Age'] >=  int(pref_age_l)) & (male_df['Age'] <=  int(pref_age_u)) &\
                                 (male_df['Partner_Age_LI'] <=  int(user_age)) & (male_df['Partner_Age_UI'] >=  int(user_age))]
    return pref_male_df

In [3]:
db  = mysql.connector.connect(host="localhost", user="root", passwd="", database="matrimony")

# prepare a cursor object using cursor() method
cursor = db.cursor()

# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")

# Fetch a single row using fetchone() method.
db_version = cursor.fetchone()

print("Database version : %s " % db_version)

Database version : 10.1.31-MariaDB 


## Let us find the matches for M5929711

In [4]:
mycursor = db.cursor()

mycursor.execute("SELECT * FROM database_for_matrimony1 where Profile_ID = 'M5929711'")

myresult      =  mycursor.fetchall()

mycol         =  ['S_No','Profile_ID','Age','Gender','Marital_Status','Caste',\
                  'Sub_Caste','State','City','Country','Height','physical_status',\
                  'Family_Status','Family_Type','Family_Values','Groom_s_Age',\
                  'Groom_s_Education','Groom_s_Income','Groom_s_Location',\
                  'About_Description']     

user_df      =   pd.DataFrame(myresult, columns= mycol)

print(user_df.head().T)  
print(list(user_df))
print(user_df.info())


                               0
S_No                           1
Profile_ID              M5929711
Age                           27
Gender                    Female
Marital_Status     Never married
Caste                   Mudaliar
Sub_Caste              Melakarar
State                 Tamil Nadu
City                     Chennai
Country                    India
Height                       5.8
physical_status           Normal
Family_Status       Middle Class
Family_Type                Joint
Family_Values           Orthodox
Groom_s_Age                28-30
Groom_s_Education       Graduate
Groom_s_Income            400000
Groom_s_Location      Tamil Nadu
About_Description             NA
['S_No', 'Profile_ID', 'Age', 'Gender', 'Marital_Status', 'Caste', 'Sub_Caste', 'State', 'City', 'Country', 'Height', 'physical_status', 'Family_Status', 'Family_Type', 'Family_Values', 'Groom_s_Age', 'Groom_s_Education', 'Groom_s_Income', 'Groom_s_Location', 'About_Description']
<class 'pandas.core.frame.

### Get all male profiles from the data base

In [5]:
mycursor = db.cursor()

mycursor.execute("SELECT * FROM database_for_matrimony1 where Gender = 'Male'")

myresult_M    =  mycursor.fetchall()

mycol         =  ['S_No','Profile_ID','Age','Gender','Marital_Status','Caste',\
                  'Sub_Caste','State','City','Country','Height','physical_status',\
                  'Family_Status','Family_Type','Family_Values','Partner_s_Age',\
                  'Partner_s_Education','Partner_s_Income','Partner_s_Location',\
                  'About_Description']     

male_df      =   pd.DataFrame(myresult_M, columns= mycol)

print(male_df.head().T)  

### Close db connection and release all resources

db.close()

                                    0              1            2  \
S_No                               13             14           15   
Profile_ID                   M5929723       M5929724     M5929725   
Age                                27             30           30   
Gender                           Male           Male         Male   
Marital_Status       Awaiting Divorce       Divorced     Divorced   
Caste                          Thevar        Maravar       Kallar   
Sub_Caste                 Mukkulathor    Mukkulathor  Mukkulathor   
State                      Tamil Nadu     Tamil Nadu   Tamil Nadu   
City                          Chennai        Chennai      Chennai   
Country                         India          India        India   
Height                            5.7            5.4          5.5   
physical_status                Normal         Normal       Normal   
Family_Status                    Rich           Rich         Rich   
Family_Type                   Nucl

In [6]:
"""
Check if the user has specified in the prefered partners age range

"""
Pref_Age_Lo = 0 # Initialize the preferred age limits
Pref_Age_Up = 0

strlength = user_df['Groom_s_Age'].str.len()[0]

if strlength == 5:
        lo, up = re.findall('\d+', str(user_df['Groom_s_Age'][0]))
else:
        lo     =  user_df['Age'] + 1
        up     =  user_df['Age'] + 5
        
Pref_Age_Lo    =  lo
Pref_Age_Up    =  up

user_df['Partner_Age_LI']   =   lo
user_df['Partner_Age_UI']   =   up

user_df['Partner_Age_LI'] = user_df['Partner_Age_LI'].astype('int32')
user_df['Partner_Age_UI'] = user_df['Partner_Age_UI'].astype('int32')


print(user_df.head().T)

                               0
S_No                           1
Profile_ID              M5929711
Age                           27
Gender                    Female
Marital_Status     Never married
Caste                   Mudaliar
Sub_Caste              Melakarar
State                 Tamil Nadu
City                     Chennai
Country                    India
Height                       5.8
physical_status           Normal
Family_Status       Middle Class
Family_Type                Joint
Family_Values           Orthodox
Groom_s_Age                28-30
Groom_s_Education       Graduate
Groom_s_Income            400000
Groom_s_Location      Tamil Nadu
About_Description             NA
Partner_Age_LI                28
Partner_Age_UI                30


In [7]:
print(male_df.shape)

(52, 20)


From these 52 male records, we need to check how many match the expectation of the specified user

In [8]:
user_age        = user_df['Age']
user_profile_ID = user_df['Profile_ID']

### Female preference

pref_age_l = user_df['Partner_Age_LI']
pref_age_u = user_df['Partner_Age_UI']

### Male preference

strlength = male_df['Partner_s_Age'].str.len()[0]

if strlength == 5:
        lo, up = re.findall('\d+', str(male_df['Partner_s_Age'][0]))
        
male_df['Partner_Age_LI']   =   int(lo)
male_df['Partner_Age_UI']   =   int(up)

### Both forward match and reverse match

pref_male_df  = male_df.loc[(male_df['Age'] >=  int(pref_age_l)) & (male_df['Age'] <=  int(pref_age_u)) &\
                                 (male_df['Partner_Age_LI'] <=  int(user_age)) & (male_df['Partner_Age_UI'] >=  int(user_age))]

print(pref_male_df.T)
    
msg = ' Count of matches: %d' % (pref_male_df['Profile_ID'].count())
   
print(msg)

                                1            2            5              6   \
S_No                            14           15           18             19   
Profile_ID                M5929724     M5929725     M5929728       M5929729   
Age                             30           30           29             29   
Gender                        Male         Male         Male           Male   
Marital_Status            Divorced     Divorced     Divorced       Divorced   
Caste                      Maravar       Kallar       Thevar         Kallar   
Sub_Caste              Mukkulathor  Mukkulathor  Mukkulathor    Mukkulathor   
State                   Tamil Nadu   Tamil Nadu   Tamil Nadu     Tamil Nadu   
City                       Chennai      Chennai      Chennai        Chennai   
Country                      India        India        India          India   
Height                         5.4          5.5          5.2            5.6   
physical_status             Normal       Normal     

In [9]:
### Display results

user                =  user_df['Profile_ID'][0]
user_age            =  user_df['Age'][0]
user_gender         =  user_df['Gender'][0]

print('Profile ID: ',   user)
print('User age : ',    user_age)
print('User gender : ', user_gender)

print('Preferred Lower limit age for partner: ', pref_age_l[0])
print('Preferred Upper limit age for partner: ', pref_age_u[0])

print('Count of matches based on user preference: ', pref_male_df['Profile_ID'].count())



Profile ID:  M5929711
User age :  27
User gender :  Female
Preferred Lower limit age for partner:  28
Preferred Upper limit age for partner:  30
Count of matches based on user preference:  19


### Relax Age limit and check the match count

In [10]:
relax_years    =  [4, 5, 6, 7] ## To get from the user
count_relax    =  [ ]
relax_age_l    =  [ ]
relax_age_u    =  [ ]

for relax in relax_years:
    
    age_l           =    pref_age_l[0] + 1
    age_u           =    pref_age_u[0] + int(relax) 

    print('\nRelaxed upper age limit is now %d - %d' %(age_l, age_u))
    df = get_matched_profiles(user_age, age_l, age_u)
    relax_count      =   df['Profile_ID'].count()  
    count_relax.append(relax_count)
    relax_age_l.append(age_l)
    relax_age_u.append(age_u)   
    
print(count_relax)


Relaxed upper age limit is now 29 - 34

Relaxed upper age limit is now 29 - 35

Relaxed upper age limit is now 29 - 36

Relaxed upper age limit is now 29 - 37
[27, 28, 30, 31]


In [11]:
# Key - Value 
Keys              =   ['Attributes', 'user', 'Age', 'Gender',  ]
Values            =   list(['Value', user, user_age, user_gender])

In [12]:
type(Values)

list

In [13]:
x    =  PrettyTable([Keys[0], '\t', Values[0]])
for i in range(1, len(Keys)):    x.add_row([Keys[i], '\t\t\t\t', Values[i]])

In [14]:
x.add_row(['Prefer Age from  ', 'Prefer Age upto  ', 'matched Count'])
x.add_row([pref_age_l[0], pref_age_u[0], pref_male_df['Profile_ID'].count()])

In [15]:
x.add_row(['relax Age from  ', 'relax Age upto  ', 'matched count'])

for i in range(0, len(count_relax)):
    x.add_row([relax_age_l[i], relax_age_u[i], count_relax[i]])

In [16]:
html_code = x.get_html_string()

In [17]:
html_file = open('table2.html','w')
html_file = html_file.write(html_code)
webbrowser.open_new_tab('table2.html')

True